In [322]:
import ds_utils
import pandas as pd
df = pd.read_csv('train.csv')

y = df['Survived']
df = df.drop(columns=['Survived'])

train_num = len(df)
df = pd.concat((df, pd.read_csv('test.csv')))

In [323]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [324]:
df['Name'].head(10)

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
5                                     Moran, Mr. James
6                              McCarthy, Mr. Timothy J
7                       Palsson, Master. Gosta Leonard
8    Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)
9                  Nasser, Mrs. Nicholas (Adele Achem)
Name: Name, dtype: object

In [325]:
df['LastName'] = df['Name'].apply(lambda x: x.split(',')[0])

In [326]:
df['LastName'].head()

0       Braund
1      Cumings
2    Heikkinen
3     Futrelle
4        Allen
Name: LastName, dtype: object

In [327]:
df['TicketDigits'] = df['Ticket'].apply(lambda x: x.split()[1] if len(x.split()) > 1 else x)

In [328]:
df['Title'] = df['Name'].apply(lambda x: x.split(',')[1].split('.')[0])

In [329]:
df['Title'].head()

0       Mr
1      Mrs
2     Miss
3      Mrs
4       Mr
Name: Title, dtype: object

In [330]:
df['FirstName'] = df['Name'].apply(lambda x: x.split(',')[1].split('.')[1].split(' ')[1])

In [331]:
df['FirstName'].head()

0       Owen
1       John
2      Laina
3    Jacques
4    William
Name: FirstName, dtype: object

In [332]:
df = df.drop(columns=['Ticket', 'PassengerId', 'Name'])

In [333]:
ds_utils.get_nan_col_proportions(df)

[('Age', 0.20091673032849502),
 ('Fare', 0.0007639419404125286),
 ('Cabin', 0.774637127578304),
 ('Embarked', 0.0015278838808250573)]

In [334]:
df['Cabin'].head(10)

0     NaN
1     C85
2     NaN
3    C123
4     NaN
5     NaN
6     E46
7     NaN
8     NaN
9     NaN
Name: Cabin, dtype: object

In [335]:
import numpy as np
df['Cabin'] = df['Cabin'].fillna(' ') 
df['CabinLetter'] = df.apply(lambda row: ' ' if row['Cabin'] == ' ' else row['Cabin'][0], axis=1)

In [336]:
# df['CabinLetter'].head()

In [337]:
df = df.drop(columns=['Cabin'])

In [338]:
import numpy as np

df['nanAge'] = df.apply(lambda row: 1 if row['Age'] == np.nan else 0, axis=1)
# age = replace_missing_with_ml(df, 'Age')['Age']

df['nanFare'] = df.apply(lambda row: 1 if row['Fare'] == np.nan else 0, axis=1)
# fare = replace_missing_with_ml(df, 'Fare')['Fare']

df['nanEmbarked'] = df.apply(lambda row: 1 if row['Embarked'] == np.nan else 0, axis=1)
# embarked = replace_missing_with_ml(df, 'Embarked', is_classify=True)['Embarked']

# df = df.drop(columns=['CabinLetter'])

df['CabinLetter'] = df['CabinLetter'].replace(to_replace=' ', value=np.nan)

df['nanCabinLetter'] = df.apply(lambda row: 1 if row['CabinLetter'] == np.nan else 0, axis=1)
df['CabinLetter'] = df.groupby('LastName')['CabinLetter'].transform(lambda x: x.fillna(x.mode()))
df['CabinLetter'] = df.groupby('Pclass')['CabinLetter'].transform(lambda x: x.fillna(x.mode()[0]))

In [339]:
df['Age'] = df.groupby('Pclass')['Age'].transform(lambda x: x.fillna(x.median()))
df['Fare'] = df.groupby('Pclass')['Fare'].transform(lambda x: x.fillna(x.median()))
df['Embarked'] = df.groupby('Pclass')['Embarked'].transform(lambda x: x.fillna(x.mode()[0]))



# #df['nanAge'] = nanAge
# df['Age'] = age

# #df['nanFare'] = nanFare 
# df['Fare'] = fare

# # df['nanEmbarked'] = nanEmbarked 
# df['Embarked'] = embarked

# # df['nanCabinLetter'] = nanCabinLetter
# # df['CabinLetter'] =cabinLetter

In [340]:
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,LastName,TicketDigits,Title,FirstName,CabinLetter,nanAge,nanFare,nanEmbarked,nanCabinLetter
0,3,male,22.0,1,0,7.2500,S,Braund,21171,Mr,Owen,F,0,0,0,0
1,1,female,38.0,1,0,71.2833,C,Cumings,17599,Mrs,John,C,0,0,0,0
2,3,female,26.0,0,0,7.9250,S,Heikkinen,3101282,Miss,Laina,F,0,0,0,0
3,1,female,35.0,1,0,53.1000,S,Futrelle,113803,Mrs,Jacques,C,0,0,0,0
4,3,male,35.0,0,0,8.0500,S,Allen,373450,Mr,William,F,0,0,0,0


In [341]:
df.loc[df['TicketDigits'].str[-1] == '.']

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,LastName,TicketDigits,Title,FirstName,CabinLetter,nanAge,nanFare,nanEmbarked,nanCabinLetter
115,3,male,21.0,0,0,7.925,S,Pekoniemi,2.,Mr,Edvard,F,0,0,0,0
173,3,male,21.0,0,0,7.925,S,Sivola,2.,Mr,Antti,F,0,0,0,0
243,3,male,22.0,0,0,7.125,S,Maenpaa,2.,Mr,Matti,F,0,0,0,0
382,3,male,32.0,0,0,7.925,S,Tikkanen,2.,Mr,Juho,F,0,0,0,0
400,3,male,39.0,0,0,7.925,S,Niskanen,2.,Mr,Juha,F,0,0,0,0
414,3,male,44.0,0,0,7.925,S,Sundman,2.,Mr,Johan,F,0,0,0,0
433,3,male,17.0,0,0,7.125,S,Kallio,2.,Mr,Nikolai,F,0,0,0,0
579,3,male,32.0,0,0,7.925,S,Jussila,2.,Mr,Eiriik,F,0,0,0,0
590,3,male,35.0,0,0,7.125,S,Rintamaki,2.,Mr,Matti,F,0,0,0,0
636,3,male,32.0,0,0,7.925,S,Leinonen,2.,Mr,Antti,F,0,0,0,0


In [342]:
df['TicketDigits'] = df['TicketDigits'].apply(lambda x: x if x[-1] != '.' else x[:-1])

In [343]:
df.loc[df['TicketDigits'].str.len() < 5]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,LastName,TicketDigits,Title,FirstName,CabinLetter,nanAge,nanFare,nanEmbarked,nanCabinLetter
10,3,female,4.00,1,1,16.7000,S,Sandstrom,9549,Miss,Marguerite,G,0,0,0,0
12,3,male,20.00,0,0,8.0500,S,Saundercock,2151,Mr,William,F,0,0,0,0
19,3,female,24.00,0,0,7.2250,C,Masselmani,2649,Mrs,Fatima,F,0,0,0,0
26,3,male,24.00,0,0,7.2250,C,Emir,2631,Mr,Farred,F,0,0,0,0
36,3,male,24.00,0,0,7.2292,C,Mamee,2677,Mr,Hanna,F,0,0,0,0
37,3,male,21.00,0,0,8.0500,S,Cann,2152,Mr,Ernest,F,0,0,0,0
39,3,female,14.00,1,0,11.2417,C,Nicola-Yarred,2651,Miss,Jamila,F,0,0,0,0
40,3,female,40.00,1,0,9.4750,S,Ahlin,7546,Mrs,Johan,F,0,0,0,0
43,2,female,3.00,1,2,41.5792,C,Laroche,2123,Miss,Simonne,F,0,0,0,0
48,3,male,24.00,2,0,21.6792,C,Samaan,2662,Mr,Youssef,F,0,0,0,0


In [344]:
df['TicketDigits'] = df['TicketDigits'].apply(lambda x: x if x[-1] != '.' else x[:-1])

In [345]:
df.loc[df['TicketDigits'].str[-1] == '.']

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,LastName,TicketDigits,Title,FirstName,CabinLetter,nanAge,nanFare,nanEmbarked,nanCabinLetter


In [346]:
df.loc[df['TicketDigits'] == 'LINE']

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,LastName,TicketDigits,Title,FirstName,CabinLetter,nanAge,nanFare,nanEmbarked,nanCabinLetter
179,3,male,36.0,0,0,0.0,S,Leonard,LINE,Mr,Lionel,F,0,0,0,0
271,3,male,25.0,0,0,0.0,S,Tornquist,LINE,Mr,William,F,0,0,0,0
302,3,male,19.0,0,0,0.0,S,Johnson,LINE,Mr,William,F,0,0,0,0
597,3,male,49.0,0,0,0.0,S,Johnson,LINE,Mr,Alfred,F,0,0,0,0


In [347]:
df['TicketDigits'] = df['TicketDigits'].apply(lambda x: 1 if x == 'LINE' else x)
df['TicketDigits'] = df['TicketDigits'].apply(lambda x: 0 if x == 'Basle' else x)

In [348]:
df.loc[df['TicketDigits'] == 'LINE']

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,LastName,TicketDigits,Title,FirstName,CabinLetter,nanAge,nanFare,nanEmbarked,nanCabinLetter


In [349]:
df['Title'].unique()

array([' Mr', ' Mrs', ' Miss', ' Master', ' Don', ' Rev', ' Dr', ' Mme',
       ' Ms', ' Major', ' Lady', ' Sir', ' Mlle', ' Col', ' Capt',
       ' the Countess', ' Jonkheer', ' Dona'], dtype=object)

In [350]:
df['Title'] = df['Title'].apply(lambda x: x.strip())
df['Title'] = df['Title'].apply(lambda x: x if x != 'Miss' else 'Ms')

In [351]:
df['Title'].unique()

array(['Mr', 'Mrs', 'Ms', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Major',
       'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess', 'Jonkheer',
       'Dona'], dtype=object)

In [352]:
df['Embarked'] = df['Embarked'].astype('category')
df['LastName'] = df['LastName'].astype('category')
df['FirstName'] = df['FirstName'].astype('category')
df['Title'] = df['Title'].astype('category')
df['TicketDigits'] = df['TicketDigits'].astype(int)
df['CabinLetter'] = df['CabinLetter'].astype('category')
df['Sex'] = df['Sex'].astype('category')
df['familySize'] = df['SibSp'] + df['Parch']
df['isAlone'] = df['familySize'].apply(lambda x: 1 if x == 0 else 0)

In [353]:
df['is_french'] = df['Title'].apply(lambda x: 1 if x == 'Mlle' or x == 'Mme' else 0)
df['is_spanish'] = df['Title'].apply(lambda x: 1 if x == 'Don' else 0)
df['is_spanish'] = df['Title'].apply(lambda x: 1 if x == 'the Countess' else 0)
df['is_religious'] = df['Title'].apply(lambda x: 1 if x == 'Rev' else 0)
df['is_belgium'] = df['Title'].apply(lambda x: 1 if x == 'Jonkheer' else 0)
military = ['Capt', 'Col', 'Major']
df['is_military'] = df['Title'].apply(lambda x: 1 if x in military else 0)
nobility = ['Don', 'Sir', 'the Countess', 'Lady']
df['is_nobility'] = df['Title'].apply(lambda x: 1 if x in nobility else 0)

In [354]:
df['FirstName'] = df['FirstName'].apply(lambda x: x[1:] if x[0] == '(' else x)

In [355]:
df2 = df.copy()

In [356]:
df.dtypes

Pclass               int64
Sex               category
Age                float64
SibSp                int64
Parch                int64
Fare               float64
Embarked          category
LastName          category
TicketDigits         int64
Title             category
FirstName           object
CabinLetter       category
nanAge               int64
nanFare              int64
nanEmbarked          int64
nanCabinLetter       int64
familySize           int64
isAlone              int64
is_french            int64
is_spanish           int64
is_religious         int64
is_belgium           int64
is_military          int64
is_nobility          int64
dtype: object

In [357]:
df = pd.get_dummies(df)

In [358]:
df.head()

,Pclass,Age,SibSp,Parch,Fare,TicketDigits,nanAge,nanFare,nanEmbarked,nanCabinLetter,...,FirstName_Yousseff,FirstName_of,CabinLetter_A,CabinLetter_B,CabinLetter_C,CabinLetter_D,CabinLetter_E,CabinLetter_F,CabinLetter_G,CabinLetter_T
0,3,22.0,1,0,7.2500,21171,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,38.0,1,0,71.2833,17599,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,3,26.0,0,0,7.9250,3101282,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,35.0,1,0,53.1000,113803,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,3,35.0,0,0,8.0500,373450,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [359]:
df_train = df[:train_num].copy()
df_train['Survived'] = y
df_test = df[train_num:].copy()

In [360]:
df_train.head()

,Pclass,Age,SibSp,Parch,Fare,TicketDigits,nanAge,nanFare,nanEmbarked,nanCabinLetter,...,FirstName_of,CabinLetter_A,CabinLetter_B,CabinLetter_C,CabinLetter_D,CabinLetter_E,CabinLetter_F,CabinLetter_G,CabinLetter_T,Survived
0,3,22.0,1,0,7.2500,21171,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,38.0,1,0,71.2833,17599,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,3,26.0,0,0,7.9250,3101282,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,1,35.0,1,0,53.1000,113803,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,3,35.0,0,0,8.0500,373450,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [361]:
df_train.to_csv('clean_train.csv', index=False)
df_test.to_csv('clean_test.csv', index=False)